In [2]:
import cv2
import numpy as np

# --- CONFIGURATION ---
IP_ADDRESS = "192.168.1.147" 
PORT = "8080"
URL = f"http://{IP_ADDRESS}:{PORT}/video"  # CHANGEABLE

MIN_AREA = 3500 # CHANGEABLE (to consider smaller o bigger shapes)

# --- GEOMETRIC TUNING ---

# 1. Line Sensitivity:
# Only shapes LINE_ASPECT_RATIO x longer than they are wide will be considered Lines
LINE_ASPECT_RATIO = 6.5 # CHANGEABLE

# 2. Contour Approximation Precision:
# Lower this to detect Circles better (keep more vertices)
# Higher this to make shapes more blocky/polygonal. (keep less vertices)
EPSILON_COEFF = 0.035 # CHANGEABLE

def get_shape_name(hull, approx, contour):
    num_vertices = len(approx)
    x, y, w, h = cv2.boundingRect(approx)
    
    # Aspect Ratio
    aspect_ratio = float(w) / h if w > h else float(h) / w
    
    # Solidity
    area_hull = cv2.contourArea(hull)
    area_cnt = cv2.contourArea(contour)
    if area_hull == 0: return "Unidentified"
    solidity = float(area_cnt) / area_hull

    # Circularity (To distinguish Circle vs Pentagon)
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0: return "Unidentified"
    circularity = 4 * np.pi * area_cnt / (perimeter * perimeter)

    # --- FILTERS ---
    if solidity < 0.8: return "Unidentified" # CHANGEABLE

    # --- CLASSIFICATION ---

    # 1. LINE DETECTION
    # A line requires the shape to be very long
    if aspect_ratio > LINE_ASPECT_RATIO:
        return "Line"

    # 2. GEOMETRIC SHAPES
    if num_vertices == 3:
        return "Triangle"

    elif num_vertices == 4:
        if 0.85 <= aspect_ratio <= 1.15: # CHANGEABLE (to split rectangles and squares)
            return "Square"
        else:
            return "Rectangle"

    elif num_vertices == 5:
        # --- SPLIT CIRCLE/PENTAGON ---
        # A perfect pentagon has circularity aprox 0.76. A circle is  aprox 1.0
        # If it has 5 vertices but is very round (high circularity), it's a bad circle, not a pentagon
        if circularity > 0.82: # CHANGEABLE
            return "Circle"
        else:
            return "Pentagon"

    elif num_vertices > 5:
        # Standard Circle Check
        if 0.7 <= circularity <= 1.2: # CHANGEABLE
            return "Circle"
    
    return "Unidentified"

def main():
    cap = cv2.VideoCapture(URL)
    cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

    kernel_noise = np.ones((1, 1), np.uint8) # CHANGEABLE
    
    # Increasing this helps lines connect better (to dilate more)
    kernel_dilate = np.ones((5, 5), np.uint8) # CHANGEABLE (Thicker lines)

    print(f"Connected to {URL}. Tuning: Lines > {LINE_ASPECT_RATIO}x, Epsilon {EPSILON_COEFF}")

    while True:
        ret, frame = cap.read()
        if not ret: break

        # Grayscale image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Blurred image
        blurred = cv2.GaussianBlur(gray, (5, 5), 0) # CHANGEABLE
        
        # Thresholding
        thresh = cv2.adaptiveThreshold(blurred, 255, # If the intensity is higher than de th, we put it in 255. Otherwise we put it in 0
                                       cv2.ADAPTIVE_THRESH_GAUSSIAN_C, # To decide if something is an important pattern or just background
                                       cv2.THRESH_BINARY_INV, # We want the pattern white and de background black
                                       19, # CHANGEABLE (Blocksize: number of neighbours to consider to decide if a pixel turns 0 or 255)
                                       16) # CHANGEABLE (Severity constant: higher means more strict, lower means more permissive)

        # Increased iterations to delete thinner lines in de erosion operation
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel_noise, iterations=2) # CHANGEABLE
        
        # Increased iterations to make lines very bold
        dilated = cv2.dilate(opening, kernel_dilate, iterations=2) # CHANGEABLE

        # This function looks for shapes of white pixels over the black background of the dilated image
        # Returns a list of the detected elements (triangles, circles, squares...)
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
        # It also returns the hierarchy of the elements, but we discard it since we do not need it 

        # Iterate through every contour (potential shape) detected in the image
        for cnt in contours:

            # Calculate the area of the contour in pixels to determine its size
            area = cv2.contourArea(cnt)
            
            # Filter out small noise (like grid dots) using the minimum area threshold
            if area > MIN_AREA:

                # Calculate the Convex Hull: The "rubber band" wrapper around the shape
                # This ignores internal dents or imperfections in the hand-drawing of the shapes
                hull = cv2.convexHull(cnt)

                # Calculate the perimeter (arc length) of the hull to set the precision scale
                peri = cv2.arcLength(hull, True)
                
                # --- APPLY NEW EPSILON ---
                # Calculate the approximation accuracy (Epsilon)
                # It determines how much we can simplify the shape
                # EPSILON_COEFF * peri defines the maximum distance between the original curve and the approximation             
                epsilon = EPSILON_COEFF * peri 
                
                # Approximate the polygonal curve (Douglas-Peucker algorithm)
                # Reduces the number of points to find the essential corners (vertices)
                approx = cv2.approxPolyDP(hull, epsilon, True)

                # Identify the shape name using our custom logic (checking vertices, aspect ratio, solidity)
                name = get_shape_name(hull, approx, cnt)
                
                # Only proceed if the shape was successfully identified
                if name != "Unidentified":

                    # Draw the Convex Hull outline in GREEN on the original frame
                    cv2.drawContours(frame, [hull], -1, (0, 255, 0), 3)
                    for p in approx:
                        # Draw RED dots on corners: vertices
                        cv2.circle(frame, (p[0][0], p[0][1]), 8, (0, 0, 255), -1)

                    # Get the bounding box coordinates (x, y, width, height) to place the text
                    x, y, w, h = cv2.boundingRect(approx)
                    # Write the name of the shape in BLUE above the bounding box
                    cv2.putText(frame, name, (x, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        # DEBUG WINDOW
        cv2.imshow("Debug: Thick Lines", dilated)
        # OUTPUT WINDOW
        cv2.imshow("Final Output", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Connected to http://192.168.1.147:8080/video. Tuning: Lines > 6.5x, Epsilon 0.035
